# **Stable Diffusion 2.1**


In [ ]:
print("Hello!")

# Install dependencies


In [ ]:
!pip install --upgrade git+https://github.com/huggingface/diffusers.git
# !pip install diffusers
!pip install --upgrade git+https://github.com/huggingface/transformers/
# !pip install transformers
!pip install accelerate==0.12.0
!pip install scipy
!pip install ftfy
!pip install gradio -q

#@markdown ### ⬅️ Run this cell
#@markdown ---
#@markdown ### Install **xformers**?
#@markdown This will take an additional ~3.5 mins.<br>But images will generate 25-40% faster.
install_xformers = False #@param {type:"boolean"}

if install_xformers:
  import os
  from subprocess import getoutput

  os.system("pip install --extra-index-url https://download.pytorch.org/whl/cu113 torch torchvision==0.13.1+cu113")
  os.system("pip install triton==2.0.0.dev20220701")
  gpu_info = getoutput('nvidia-smi')
  if("A10G" in gpu_info):
      os.system(f"pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl")
  elif("T4" in gpu_info):
      os.system(f"pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+1515f77.d20221130-cp38-cp38-linux_x86_64.whl")


# ### install xformers
# from IPython.utils import capture
# from subprocess import getoutput
# from re import search

# with capture.capture_output() as cap:

#     smi_out = getoutput('nvidia-smi')
#     supported = search('(T4|P100|V100|A100|K80)', smi_out)

#     if not supported:
#       while True:
#         print("\x1b[1;31mThe current GPU is not supported, try starting a new session.\x1b[0m")
#     else:
#       supported = supported.group(0)

# !pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/{supported}/xformers-0.0.13.dev0-py3-none-any.whl
# !pip install -q https://github.com/ShivamShrirao/xformers-wheels/releases/download/4c06c79/xformers-0.0.15.dev0+4c06c79.d20221201-cp38-cp38-linux_x86_64.whl

# Run the app

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras
import layers, models
from sklearn.model_selection import train_test_split

# Load the preprocessed medical images and labels
X = np.load('medical_images.npy')
y = np.load('labels.npy')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Capsule Network architecture
class CapsuleNetwork(models.Model):
    def _init_(self, input_shape, num_classes):
        super(CapsuleNetwork, self)._init_()

        # Convolutional layer
        self.conv1 = layers.Conv2D(filters=256, kernel_size=9, padding='valid', activation='relu', input_shape=input_shape)

        # Primary Capsules layer
        self.primary_capsules = layers.Conv2D(filters=256, kernel_size=9, strides=2, padding='valid')

        # Digit Capsules layer
        self.digit_capsules = layers.Dense(units=num_classes * 16, activation=None)

        # Reshape Digit Capsules to have a shape of [batch_size, num_classes, 16]
        self.digit_capsules_reshape = layers.Reshape(target_shape=[num_classes, 16])

        # Squash activation function
        self.squash = Squash()

    def call(self, inputs):
        # Convolutional layer
        x = self.conv1(inputs)

        # Primary Capsules layer
        x = self.primary_capsules(x)
        x = self.squash(x)

        # Digit Capsules layer
        x = self.digit_capsules(x)
        x = self.digit_capsules_reshape(x)
        x = self.squash(x)

        # Length of Capsule vectors
        lengths = tf.sqrt(tf.reduce_sum(tf.square(x), axis=-1))

        return lengths

# Squash activation function
class Squash(layers.Layer):
    def call(self, inputs):
        norm = tf.norm(inputs, axis=-1, keepdims=True)
        squared_norm = norm ** 2
        scale = squared_norm / (1 + squared_norm)
        return scale * inputs / norm

# Compile the model
model = CapsuleNetwork(input_shape=(128, 128, 3), num_classes=2)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:' ,score[1])
